In [1]:
import os
os.chdir(os.path.pardir)
from dataset.dataset import Dataset
from evaluation_metrics.diversity_metrics import Topic_diversity
from evaluation_metrics.topic_significance_metrics import KL_uniform
from skopt import gp_minimize, forest_minimize, dummy_minimize
from optimization.optimizer import Optimizer
from skopt.space.space import Real, Integer
import multiprocessing as mp
from models.TorchAvitm import TorchAvitm
from gensim.models import Word2Vec

In [2]:
# Load dataset
dataset = Dataset()
dataset.load("preprocessed_datasets/newsgroup/newsgroup_lemmatized_5")

True

In [3]:
# Load model
model = TorchAvitm()

In [4]:

# Set model hyperparameters
model.hyperparameters['prior_variance'] = 0.2
model.hyperparameters['n_components'] = 5
model.hyperparameters['num_epochs'] = 1

In [5]:
model.hyperparameters


{'prior_variance': 0.2, 'n_components': 5, 'num_epochs': 1}

In [6]:
model.test_set(True)

In [7]:
model.train_model(dataset, model.hyperparameters)

Settings: 
               N Components: 5
               Topic Prior Mean: 0.0
               Topic Prior Variance: 0.2
               Model Type: prodLDA
               Hidden Sizes: (100, 100)
               Activation: softplus
               Dropout: 0.2
               Learn Priors: True
               Learning Rate: 0.002
               Momentum: 0.99
               Reduce On Plateau: False
               Save Dir: None
Epoch: [1/1]	Samples: [10840/10840]	Train Loss: 16979.011972440036	Time: 0:00:01.573792


{'topics': [['able',
   'absolutely',
   'absolute',
   'absurd',
   'large',
   'consumer',
   'connector',
   'cult',
   'brief',
   'sender'],
  ['translation',
   'translate',
   'waste',
   'notion',
   'town',
   'sample',
   'sign',
   'early',
   'sun',
   'reach'],
  ['workstation',
   'frequency',
   'sure',
   'involve',
   'quick',
   'purely',
   'pound',
   'font',
   'million',
   'arise'],
  ['orient',
   'ability',
   'sun',
   'capable',
   'translation',
   'translate',
   'wait',
   'sign',
   'easily',
   'bunch'],
  ['orient',
   'waste',
   'translate',
   'translation',
   'ability',
   'shareware',
   'sign',
   'component',
   'acknowledge',
   'photo']],
 'topic-word-matrix': array([[-0.16706164,  0.12810463,  0.07583538, ..., -0.12239905,
         -0.14110298, -0.13873254],
        [ 0.02208466,  0.01684416,  0.05748036, ...,  0.06680206,
          0.12051603, -0.13030963],
        [ 0.01077029,  0.01355652, -0.00893499, ...,  0.14869225,
         -0.1157478

In [8]:
prova = model.inference()

In [9]:
len(model.info_test())

7208

In [ ]:
# Topic diversity
topic_diversity = Topic_diversity()

# KL_Uniform
#kl_uniform = KL_uniform()

In [ ]:
# Define optimization parameters
opt_params = {}
opt_params["n_calls"] = 30
opt_params["minimizer"] = forest_minimize
opt_params["different_iteration"] = 3
opt_params["n_random_starts"] = 5
#opt_params["extra_metrics"] = [kl_uniform] # List of extra metrics
opt_params["n_jobs"] = mp.cpu_count() -1 # Enable multiprocessing
opt_params["verbose"] = True
opt_params["save_path"] = "results" #create folder if it doesn't exist 

In [ ]:
# Create search space for optimization
search_space = {
    "num_epochs": Integer(low=1, high=50),
    #"eta": Real(low=0.01, high=5.0)
}

In [ ]:
# Initialize optimizer
optimizer = Optimizer(
    model,
    dataset,
    topic_diversity,
    search_space,
    opt_params)

In [ ]:
# Disable computing of topic document matrix to optimize performance
optimizer.topic_document_matrix = False
optimizer.topic_word_matrix = False

In [ ]:
# Optimize
res = optimizer.optimize()

print(res.hyperparameters) # Best values for the hyperparameters
print(res.function_values) # Score of the optimized metric
print("Optimized metric: "+res.optimized_metric)

40